In [1]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Tensorflow version 2.5.0
Running on TPU  ['10.78.220.202:8470']
INFO:tensorflow:Initializing the TPU system: grpc://10.78.220.202:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.78.220.202:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [2]:
!pip install -q tensorflow numpy pandas dask matplotlib seaborn sklearn gym tqdm

In [3]:
!pip install -q git+https://github.com/schr476/HUGS2021.git#egg=hugs2021 

In [4]:
from hugs2021 import test
test.hello_hugs2021()

Hello HUGS2021


In [5]:
# Train
import time
import gym
EPISODES = 250
NSTEPS: int = 100
# Setup environment
estart = time.time()
env_version = 1
env = gym.make('CartPole-v1')
env._max_episode_steps = NSTEPS
env.seed(1)
end = time.time()
print('Time init environment: %s' % str((end - estart) / 60.0))
print('Using environment: %s' % env)
print('Observation_space: %s' % env.observation_space.shape)
print('Action_size: %s' % env.action_space)

Time init environment: 0.0001935243606567383
Using environment: <TimeLimit<CartPoleEnv<CartPole-v1>>>
Observation_space: 4
Action_size: Discrete(2)


In [6]:
from hugs2021.dqn import DQN
arch_type = 'MLP'
nmodels = 1
agent = DQN(env, arch_type=arch_type, nmodels=nmodels)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense (Dense)                (None, 64)                320       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 130       
Total params: 8,770
Trainable params: 8,770
Non-trainable params: 0
_________________________________________________________________
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape 

In [ ]:
from tqdm import tqdm
global_counter  =0
episode_total_rewards = []
for e in tqdm(range(EPISODES), desc='RL Episodes', leave=True):
    current_state = env.reset()
    total_reward = 0
    done = False
    step_counter = 0
    while not done:
      action, policy_type = agent.action(current_state)
      next_state, reward, done, _ = env.step(action)
      agent.remember(current_state, action, reward, next_state, done)
      if global_counter >= 25:# and global_counter % 2 == 0:
        agent.train()
      current_state = next_state
      # Increment total reward
      total_reward += reward
      step_counter += 1
      global_counter+1
      # Check if maximum episode steps is reached
      if step_counter >= NSTEPS:
        done = True
    episode_total_rewards.append(total_reward)

RL Episodes:  28%|██▊       | 69/250 [00:26<01:17,  2.32it/s]

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
fig= plt.figure(figsize=(8,6),dpi=100)
frame = pd.DataFrame(episode_total_rewards)
frame['total_reward_roll'] = frame.rolling(25).mean()
plt.plot(frame['total_reward_roll'])